In [ ]:
# Name : Priyanka Manoj Naik - 50248591

In [3]:
import tweepy
from tweepy import OAuthHandler
import json
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import time
import os
import sys

'''Function that loads the twitter API after authorizingthe user.'''
def load_api():
    consumer_key = "d9q3JVpPEOhb7pLqlP3Tn7LBw"
    consumer_secret = "aTZj72mUPrUvZHOofDzOAMp8Ro1FVwHNzEKPZA4H3kww13RhKJ"
    access_token = "960608039458963456-Zt65NRJyDL4ws97zsOND8lPpLdXweL8"
    access_secret = "KjFj1n7HryIxm4MmjuF3df5hNXmmT1vecqZGokYfPWkMV"
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

''' Function that takes in a search string 'query', the maximum
    number of tweets 'max_tweets', and the minimum (i.e., starting)
    tweet id. It returns a list of tweepy.models.Status objects. '''
def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets, since_id=str(since_id), max_id=str(max_id-1), geocode = geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now() + dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id

''' Function that gets the ID of a tweet. This ID can
    then be used as a 'starting point' from which to
    search. The query is required and has been set to
    a commonly used word by default. The variable
    'days_ago' has been initialized to the maximum amount
    we are able to search back in time (9).'''
def get_tweet_id(api, date='', days_ago=9, query='a'): 
    if date: # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id
    
''' Function that appends tweets to a file. '''
def write_tweets(tweets, filename):
    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')
            
def main():
    search_phrases = ['immigration']
    time_limit = 1.5 # runtime limit in hours
    max_tweets = 100                           # number of tweets per search (will be
                                               # iterated over) - maximum is 100
    min_days_old, max_days_old = 1, 2          # search limits e.g., from 7 to 8
                                               # gives current weekday from last week,
                                               # min_days_old=0 will search from right now
    USA = '39.8,-95.583068847656,2500km'       # this geocode includes nearly all American
                                               # states (and a large portion of Canada)
    
    tweet_files = []
    # loop over search items,
    # creating a new file for each
    for search_phrase in search_phrases:

        print('Search phrase =', search_phrase)

        ''' other variables '''
        name = search_phrase.split()[0]
        json_file_root = 'TwitterData/' + name
        try:
            if not os.path.exists(os.path.dirname(json_file_root)):
                os.makedirs(os.path.dirname(json_file_root))
        except OSError as err:
            print(err)
        read_IDs = False
        
        # open a file in which to store the tweets
        if max_days_old - min_days_old == 1:
            d = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        else:
            d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
            d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
        json_file = json_file_root + '_' + day + '.json'
        tweet_files.append(json_file)
        if os.path.isfile(json_file):
            print('Appending tweets to file named: ',json_file)
            read_IDs = True
        
        # authorize and load the twitter API
        api = load_api()
        
        # set the 'starting point' ID for tweet collection
        if read_IDs:
            # open the json file and get the latest tweet ID
            with open(json_file, 'r') as f:
                lines = f.readlines()
                max_id = json.loads(lines[-1])['id']
                print('Searching from the bottom ID in file')
        else:
            # get the ID of a tweet that is min_days_old
            if min_days_old == 0:
                max_id = -1
            else:
                max_id = get_tweet_id(api, days_ago=(min_days_old-1))
        # set the smallest ID to search for
        since_id = get_tweet_id(api, days_ago=(max_days_old-1))
        print('max id (starting point) =', max_id)
        print('since id (ending point) =', since_id)
        
        ''' tweet gathering loop  '''
        start = dt.datetime.now()
        end = start + dt.timedelta(hours=time_limit)
        count, exitcount = 0, 0
        while dt.datetime.now() < end:
            count += 1
            print('count =',count)
            # collect tweets and update max_id
            tweets, max_id = tweet_search(api, search_phrase, max_tweets,max_id=max_id, since_id=since_id,geocode=USA)
            # write tweets to file in JSON format
            if tweets:
                write_tweets(tweets, json_file)
                exitcount = 0
            else:
                exitcount += 1
                if exitcount == 3:
                    if search_phrase == search_phrases[-1]:
                        sys.exit('Maximum number of empty tweet strings reached - exiting')
                    else:
                        print('Maximum number of empty tweet strings reached - breaking')
                        break

if __name__ == "__main__":
    main()

('Search phrase =', 'immigration')
('search limit (start/stop):', datetime.datetime(2018, 4, 4, 23, 59, 59))
('search limit (start/stop):', datetime.datetime(2018, 4, 3, 23, 59, 59))
('max id (starting point) =', 981682829506605067)
('since id (ending point) =', 981320441649229824)
('count =', 1)
('found', 100, 'tweets')
('count =', 2)
('found', 100, 'tweets')
('count =', 3)
('found', 100, 'tweets')
('count =', 4)
('found', 100, 'tweets')
('count =', 5)
('found', 100, 'tweets')
('count =', 6)
('found', 100, 'tweets')
('count =', 7)
('found', 100, 'tweets')
('count =', 8)
('found', 100, 'tweets')
('count =', 9)
('found', 100, 'tweets')
('count =', 10)
('found', 100, 'tweets')
('count =', 11)
('found', 100, 'tweets')
('count =', 12)
('found', 100, 'tweets')
('count =', 13)
('found', 100, 'tweets')
('count =', 14)
('found', 100, 'tweets')
('count =', 15)
('found', 100, 'tweets')
('count =', 16)
('found', 100, 'tweets')
('count =', 17)
('found', 100, 'tweets')
('count =', 18)
('found', 100

('found', 100, 'tweets')
('count =', 189)
('found', 100, 'tweets')
('count =', 190)
('found', 100, 'tweets')
('count =', 191)
('found', 100, 'tweets')
('count =', 192)
('found', 100, 'tweets')
('count =', 193)
('found', 100, 'tweets')
('count =', 194)
('found', 100, 'tweets')
('count =', 195)
('found', 100, 'tweets')
('count =', 196)
('found', 100, 'tweets')
('count =', 197)
('found', 100, 'tweets')
('count =', 198)
('found', 100, 'tweets')
('count =', 199)
('found', 100, 'tweets')
('count =', 200)
('found', 100, 'tweets')
('count =', 201)
('found', 100, 'tweets')
('count =', 202)
('found', 100, 'tweets')
('count =', 203)
('found', 100, 'tweets')
('count =', 204)
('found', 100, 'tweets')
('count =', 205)
('found', 100, 'tweets')
('count =', 206)
('found', 100, 'tweets')
('count =', 207)
('found', 100, 'tweets')
('count =', 208)
('found', 100, 'tweets')
('count =', 209)
('found', 100, 'tweets')
('count =', 210)
('found', 100, 'tweets')
('count =', 211)
('found', 100, 'tweets')
('count =

('found', 100, 'tweets')
('count =', 376)
('found', 100, 'tweets')
('count =', 377)
('found', 100, 'tweets')
('count =', 378)
('found', 100, 'tweets')
('count =', 379)
('found', 100, 'tweets')
('count =', 380)
('found', 100, 'tweets')
('count =', 381)
('found', 100, 'tweets')
('count =', 382)
('found', 100, 'tweets')
('count =', 383)
('found', 100, 'tweets')
('count =', 384)
('found', 100, 'tweets')
('count =', 385)
('found', 100, 'tweets')
('count =', 386)
('found', 100, 'tweets')
('count =', 387)
('found', 100, 'tweets')
('count =', 388)
('found', 100, 'tweets')
('count =', 389)
('found', 100, 'tweets')
('count =', 390)
('found', 100, 'tweets')
('count =', 391)
('found', 100, 'tweets')
('count =', 392)
('found', 100, 'tweets')
('count =', 393)
('found', 100, 'tweets')
('count =', 394)
('found', 100, 'tweets')
('count =', 395)
('found', 100, 'tweets')
('count =', 396)
('found', 100, 'tweets')
('count =', 397)
('found', 100, 'tweets')
('count =', 398)
('found', 100, 'tweets')
('count =

SystemExit: Maximum number of empty tweet strings reached - exiting

/Users/priyankanaik/Library/Python/2.7/lib/python/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import urllib2
import json
import datetime
import time
import sys, os
import logging
from urllib2 import HTTPError
from ConfigParser import SafeConfigParser


# helper function to iterate through dates
def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )

# helper function to get json into a form I can work with       
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

# helpful function to figure out what to name individual JSON files        
def getJsonFileName(date, page, json_file_path, query):
    json_file_name = "_".join([query, date, str(page)])
    json_file_name = "".join([json_file_name, ".", 'json'])
    json_file_name = "".join([json_file_path,json_file_name])
    return json_file_name

# helpful function for processing keywords, mostly    
def getMultiples(items, key):
    values_list = ""
    if len(items) > 0:
        num_keys = 0
        for item in items:
            if num_keys == 0:
                values_list = item[key]                
            else:
                values_list =  "; ".join([values_list,item[key]])
            num_keys += 1
    return values_list
    
# get the articles from the NYTimes Article API    
def getArticles(start_date, end_date, query, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(101):
        for n in range(5): # 5 tries
            try:
                request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=" + query + "&begin_date=" + start_date + "&end_date=" + end_date + "&page=" + str(page) + "&api-key=" + api_key
                response = urllib2.urlopen(request_string)
                content = response.read()
                if content:
                    articles = convert(json.loads(content))
                    # if there are articles here
                    if len(articles["response"]["docs"]) >= 1:
                        json_file_name = getJsonFileName(start_date, page, json_file_path, query)
                        json_file = open(json_file_name, 'w')
                        json_file.write(content)
                        json_file.close()
                    # if no more articles, go to next date
                    else:
                        return
                time.sleep(3) # wait so we don't overwhelm the API
            except HTTPError as e:
                logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, start_date, e.code, e.reason, request_string)
                if e.code == 403:
                    print "Script hit a snag and got an HTTPError 403. Check your log file for more info."
                    return
                if e.code == 429:
                    print "Waiting. You've probably reached an API limit."
                    time.sleep(30) # wait 30 seconds and try again
            except: 
                logging.error("Error on %s page %s: %s", start_date, file_number, sys.exc_info()[0])
                continue
        
# Main function where stuff gets done
def main():
    log_file = "NewsData/logfile.log"
    api_key = "67736f1f941b44dc95f63619f6bdc9fb"
    start = datetime.date(year=2018, month=4, day=4)
    end = datetime.date(year=2018,month=4, day=4) 
    query = 'immigration'
    logging.basicConfig(filename=log_file, level=logging.INFO)
    
    logging.info("Getting started.") 
    try:
        # LOOP THROUGH THE SPECIFIED DATES
        for date in daterange(start, end):
            start_date = date.strftime("%Y%m%d")
            print start_date            
            date += datetime.timedelta(days=1)
            end_date = date.strftime("%Y%m%d")
            print end_date
            logging.info("Working on %s." % start_date)
            json_file_path = "NewsData/"
            try:
                if not os.path.exists(os.path.dirname(json_file_path)):
                    os.makedirs(os.path.dirname(json_file_path))
            except OSError as err:
                print(err)
            getArticles(start_date, end_date, query, api_key, json_file_path)
    except:
        logging.error("Unexpected error: %s", str(sys.exc_info()[0]))
    finally:
        logging.info("Finished.")

if __name__ == '__main__' :
    main()

20180404
20180405


In [94]:
'''mapper.py'''
import os, json, sys
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import operator 
from collections import Counter
from nltk.corpus import stopwords
import string

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

def get_json_files(path_to_json):
    tweet_files = []
    json_path = "".join([path_to_json, "/"])
    tweet_files.append([str(json_path + pos_json) for pos_json in os.listdir(json_path) if pos_json.endswith('.json')])
    return tweet_files


def read_json_files(tweet_files):
    tweets = []
    for tweet_file in tweet_files:
        for fil in tweet_file:
            with open(str(fil), 'r') as f:
                for line in f.readlines():
                    tweets.append(json.loads(line))
    return tweets

def read_input(file):
    tweets = []
    for line in file:
        tweets.append(json.loads(line))
    return tweets


def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stop]
    return filtered_tokens

def mapper(tweets):
    tokens = []
    terms_only = []
    d = []
    for tweet in tweets[:5]:
        tokens.extend(preprocess(tweet['text']))
        # Count terms only (no hashtags, no mentions)
        terms_only.extend([term for term in preprocess(tweet['text']) if not term.startswith(('#', '@'))])
    for term in terms_only:
        d.append((term.encode('utf-8'), 1))
    return d

def main():
#     tweet_files = get_json_files('TwitterData/')
#     tweets = read_json_files(tweet_files)
    tweets = read_input(sys.stdin)
    d = mapper(tweets)
    print '\n'.join('%s %d' % val for val in d)
#     with open('mapper-output.txt', 'w') as fp:
#         fp.write('\n'.join('%s %d' % val for val in d))

if __name__ == "__main__":
    main()

huge 1
breakthrough 1
brexit 1
vote 1
british 1
public 1
utterly 1
fed 1
… 1
https://t.co/mrb1ka0rh5 1
build 1
wall 1
deport 1
every 1
illegal 1
prisons 1
end 1
chain 1
migration 1
make 1
english 1
official 1
language 1
us 1
… 1
oakland 1
mayor 1
consulted 1
illegal-immigration 1
activists 1
tipping 1
ice 1
raid 1
https://t.co/pw9kaxl4zm 1
criminal 1
alien 1
defending 1
oakland 1
mayor 1
consulted 1
illegal-immigration 1
activists 1
warning 1
ice 1
raids 1
https://t.co/deillocopf 1
39 1
arrests 1
crackdown 1
albanian 1
crime 1
rings 1
facilitating 1
illegal 1
immigration 1
https://t.co/tprcop1rim 1


In [52]:
'''reducer.py'''
import json

def reducer(file_name):
    if os.path.isfile(file_name):
        in_file = open(file_name, 'r')
        d = {}
        for line in in_file: 
            line = line.strip()
            word, count = line.split(" ", 1)
            try:
                d[word] += count
            except:
                d[word] = count
        in_file.close()
        s = json.dumps([{'term': word, 'freq': count} for word, count in d.items()], indent = 4)
        print s
#         with open('result.json', 'w') as f:
#             f.write(s)
            
def main():
    reducer(sys.argv[1])
    
if __name__ == "__main__":
    main()

In [72]:
'''mapper-co.py'''
import os, json
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import operator 
from collections import Counter
from nltk.corpus import stopwords
import string
from nltk import bigrams 
from collections import defaultdict

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

def get_json_files(path_to_json):
    tweet_files = []
    json_path = "".join([path_to_json, "/"])
    tweet_files.append([str(json_path + pos_json) for pos_json in os.listdir(json_path) if pos_json.endswith('.json')])
    return tweet_files


def read_json_files(tweet_files):
    tweets = []
    for tweet_file in tweet_files:
        for fil in tweet_file:
            with open(str(fil), 'r') as f:
                for line in f.readlines():
                    tweets.append(json.loads(line))
    return tweets


def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stop]
    return filtered_tokens

def mapper(tweets):
    tokens = []
    terms_only = []
    com = defaultdict(lambda : defaultdict(int))
    for tweet in tweets[:5]:
        tokens.extend(preprocess(tweet['text']))
        # Count terms only (no hashtags, no mentions)
        terms_only.extend([term for term in preprocess(tweet['text']) if not term.startswith(('#', '@'))])
    # Build co-occurrence matrix
    for i in range(len(terms_only)-1):            
        for j in range(i+1, len(terms_only)):
            w1, w2 = sorted([terms_only[i].encode('utf-8'), terms_only[j].encode('utf-8')])                
            if w1 != w2:
                com[w1][w2] += 1
    return com

def main():
    tweet_files = get_json_files(sys.argv[1])
    tweets = read_json_files(tweet_files)
    com = mapper(tweets)
#     with open('mapper-output-co.txt', 'w') as fp:
    for k,v in com.iteritems():   
        for key,val in v.items():
            t = (k, (key, val))
            print '{} {} {}'.format(t[0], t[1][0], t[1][1])
#                 fp.write('{} {} {}'.format(t[0], t[1][0], t[1][1]))
#                 fp.write('\n')

if __name__ == "__main__":
    main()

In [90]:
'''reducer-co.py'''
import json

def reducer(file_name):
    if os.path.isfile(file_name):
        in_file = open(file_name, 'r')
        co_oc = defaultdict(lambda : defaultdict(int))
        for line in in_file: 
            line = line.strip()
            word, co_word, count = line.split(" ", 2)
            co_oc[word][co_word] += int(count)
        in_file.close()
        com_max = []
        # For each term, look for the most common co-occurrent terms
        for t1 in co_oc:
            t1_max_terms = sorted(co_oc[t1].items(), key=operator.itemgetter(1), reverse=True)[:10]
            for t2, t2_count in t1_max_terms:
                com_max.append(((t1, t2), t2_count))
        # Get the most frequent co-occurrences
        terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
#         with open('reducer-co.txt', 'w') as fp:
        for t in terms_max[:10]:
            print '{} {} {}'.format(t[0][0], t[0][1], t[1])
#                 fp.write('{} {} {}'.format(t[0][0], t[0][1], t[1]))
#                 fp.write('\n')

def main():
    reducer(sys.argv[1])
    
if __name__ == "__main__":
    main()

[(('activists', 'illegal-immigration'), 4), (('activists', 'consulted'), 4), (('activists', 'oakland'), 4), (('activists', 'mayor'), 4), (('activists', 'illegal'), 4), (('activists', 'ice'), 4), (('activists', '\xe2\x80\xa6'), 4), (('illegal-immigration', '\xe2\x80\xa6'), 4), (('illegal-immigration', 'oakland'), 4), (('illegal-immigration', 'mayor'), 4), (('consulted', 'illegal-immigration'), 4), (('consulted', 'oakland'), 4), (('consulted', 'mayor'), 4), (('consulted', 'illegal'), 4), (('consulted', 'ice'), 4), (('consulted', '\xe2\x80\xa6'), 4), (('oakland', '\xe2\x80\xa6'), 4), (('mayor', '\xe2\x80\xa6'), 4), (('mayor', 'oakland'), 4), (('illegal', 'illegal-immigration'), 4), (('illegal', '\xe2\x80\xa6'), 4), (('illegal', 'oakland'), 4), (('illegal', 'mayor'), 4), (('ice', 'illegal-immigration'), 4), (('ice', 'illegal'), 4), (('ice', '\xe2\x80\xa6'), 4), (('ice', 'oakland'), 4), (('ice', 'mayor'), 4), (('facilitating', 'illegal-immigration'), 2), (('facilitating', 'oakland'), 2), ((

In [ ]:
# parse the JSON files you stored into a comma-delimited file
def parseArticles(start_date, end_date, query, csv_file_name, json_file_path):
    for file_number in range(101):
        # get the articles and put them into a dictionary
        try:
            file_name = getJsonFileName(start_date, file_number, json_file_path, query)
            if os.path.isfile(file_name):
                in_file = open(file_name, 'r')
                articles = convert(json.loads(in_file.read()))
                in_file.close()
            else:
                break
        except IOError as e:
            logging.error("IOError in %s page %s: %s %s", start_date, file_number, e.errno, e.strerror)
            continue
        
        # if there are articles in that document, parse them
        if len(articles["response"]["docs"]) >= 1:  

            # open the csv for appending
            try:
                out_file = open(csv_file_name, 'ab')

            except IOError as e:
                logging.error("IOError: %s %s %s %s", start_date, file_number, e.errno, e.strerror)
                continue
        
            # loop through the articles putting what we need in a tsv   
            try:
                for article in articles["response"]["docs"]:
                    # if (article["source"] == "The New York Times" and article["document_type"] == "article"):
                    keywords = ""
                    keywords = getMultiples(article["keywords"],"value")
    
                    # should probably pull these if/else checks into a module
                    variables = [
                        article["pub_date"], 
                        keywords, 
                        str(article["headline"]["main"]).decode("utf8").replace("\n","") if "main" in article["headline"].keys() else "", 
                        str(article["source"]).decode("utf8") if "source" in article.keys() else "", 
                        str(article["document_type"]).decode("utf8") if "document_type" in article.keys() else "", 
                        article["web_url"] if "web_url" in article.keys() else "",
                        str(article["news_desk"]).decode("utf8") if "news_desk" in article.keys() else "",
                        str(article["section_name"]).decode("utf8") if "section_name" in article.keys() else "",
                        str(article["snippet"]).decode("utf8").replace("\n","") if "snippet" in article.keys() else "",
                        str(article["lead_paragraph"]).decode("utf8").replace("\n","") if "lead_paragraph" in article.keys() else "",
                        ]
                    line = ",".join(variables)
                    out_file.write(line.encode("utf8")+"\n")
            except KeyError as e:
                logging.error("KeyError in %s page %s: %s %s", start_date, file_number, e.errno, e.strerror)
                continue
            except (KeyboardInterrupt, SystemExit):
                raise
            except: 
                logging.error("Error on %s page %s: %s", start_date, file_number, sys.exc_info()[0])
                continue
        
            out_file.close()
        else:
            break

In [38]:
'''mapper.py'''
import os, json, sys
from nltk.tokenize import word_tokenize
import re
import operator
from collections import Counter
from nltk.corpus import stopwords
import string

emoticons_str = r"""
    (?:
    [:=;] # Eyes
    [oO\-]? # Nose (optional)
    [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = [
             emoticons_str,
             r'<[^>]+>', # HTML tags
             r'(?:@[\w_]+)', # @-mentions
             r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
             r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
             r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
             r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
             r'(?:[\w_]+)', # other words
             r'(?:\S)' # anything else
             ]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

def read_input(file):
    articles = []
    with open(file, "r") as f:
        for line in f:
            articles.append(json.loads(line))
    return articles

def parse_articles(articles):
    news = []
    print articles
    if len(articles) >= 1:  
        print articles
        for article in articles:
            for art in article["response"]["docs"]:
                # if (article["source"] == "The New York Times" and article["document_type"] == "article"):
                keywords = ""
                keywords = getMultiples(art["keywords"],"value")
                variables = str(art["snippet"]) if "snippet" in art.keys() else ""
                print variables
#         news.append(dic)
    return(news) 

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stop]
    return filtered_tokens

def mapper(tweets):
    tokens = []
    d = []
    for tweet in tweets:
        print tweet
#         tokens.extend(preprocess(tweet['snippet']))
#     for term in tokens:
#         d.append((term.encode('utf-8'), 1))
#     print tokens
    return d

def main():
    articles = read_input("NewsData/immigration_20170721_0.json")
    news = parse_articles(articles)
    print news
#     d = mapper(news)
#    for t in d:
#        print t[0], t[1]

if __name__ == "__main__":
    main()

In [16]:
import os, json

def get_json_files(path_to_json):
    tweet_files = []
    json_path = "".join([path_to_json, "/"])
    tweet_files.append([str(json_path + pos_json) for pos_json in os.listdir(json_path) if pos_json.endswith('.json')])
    return tweet_files


def read_json_files(tweet_files):
    tweets = []
    for tweet_file in tweet_files:
        for fil in tweet_file:
            with open(str(fil), 'r') as f:
                for line in f.readlines():
                    tweets.extend(json.loads(line))
    return tweets

def main():
    tweet_files = get_json_files("Data/")
    tweets = read_json_files(tweet_files)
    d = {}
    for tweet in tweets:
        try:
            d[tweet["term"]] += int(tweet["freq"])
        except:
            d[tweet["term"]] = int(tweet["freq"])
    s = json.dumps([{"term": word, "freq": count} for word, count in d.items()], indent = 4)
    with open("result1.json", "w") as f:
        f.write(s)
            

if __name__ == "__main__":
    main()

In [22]:
import os, json
from collections import defaultdict
import operator

def get_files(path):
    tweet_files = []
    json_path = "".join([path, "/"])
    tweet_files.append([str(json_path + pos_json) for pos_json in os.listdir(json_path) if pos_json.endswith('.txt')])
    return tweet_files


def read_files(tweet_files):
    co_oc = defaultdict(lambda : defaultdict(int))
    for tweet_file in tweet_files:
        for fil in tweet_file:
            with open(str(fil), 'r') as f:
                for line in f.readlines():
                    line = line.decode('utf8').strip()
                    word, co_word, count = line.split(" ", 2)
                    co_oc[word][co_word] += int(count)
    com_max = []
    # For each term, look for the most common co-occurrent terms
    for t1 in co_oc:
        t1_max_terms = sorted(co_oc[t1].items(), key=operator.itemgetter(1), reverse=True)[:10]
        for t2, t2_count in t1_max_terms:
            com_max.append(((t1, t2), t2_count))
    # Get the most frequent co-occurrences
    terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
    with open('result2.json', 'w') as fp:
        fp.write("[")
        for t in terms_max[:10]:
            json.dump({'term': t[0][0], 'co_term': t[0][1], 'freq': t[1]}, fp)
            fp.write(",\n")
        fp.write("]")

def main():
    tweet_files = get_files("OutputNewsDataCo/")
    tweets = read_files(tweet_files)
            

if __name__ == "__main__":
    main()